# Lab 1 - Information Security

<ul>
  <li>Andrea Andreozzi (2163406)</li>
  <li>Riccardo Scalco (2155352)</li>
  <li>Sergio Cibecchini (2155353)</li>
  <li>Luca Ferrari (2166294)</li>
</ul>

Some of the following script require the file given us to find the keys. If the files aren't in the same folder of the scirpt, add the path without the file name (for example: path="/content/").

In [114]:
path = ""

# Variable environment
Fisrt of all we define some useful and we import numpy to use some useful functions with array

In [115]:
import numpy as np
import random

In [116]:
n = 5 # number of round
p = 11 # every number in this lab are in mod p
multiplicative_value_of_f = 2 # number that is multiply in the substitution function of AES
matrix_A = np.array([[2, 5], [1, 7]])
dim = 8 # length of the key and the dimension of the squared matrix. In the last task it becomes 4
max_distance = 40 # max distance for brute force

We also define a method to read the pairs plaintext-ciphertext given in a file

In [117]:
# Function to read plaintext-ciphertext pairs from a file
def read_pairs_from_file(filepath):
    # Open the file at the given filepath in read mode
    with open(filepath, 'r') as f:
        # Read all lines from the file, stripping whitespace and ignoring empty lines
        lines = [line.strip() for line in f if line.strip()]

    # Initialize empty lists to store plaintexts and ciphertexts
    plaintexts, ciphertexts = [], []
    for line in lines:
        try:
            # Remove square brackets from the line
            line = line.replace('[', '').replace(']', '')
            # Split the line into two parts: plaintext (left) and ciphertext (right)
            left, right = line.split()  # Separate plaintext and ciphertext
            # Convert the plaintext (left) into a NumPy array of integers
            u = np.array(list(map(int, left.split(','))))
            # Convert the ciphertext (right) into a NumPy array of integers
            x = np.array(list(map(int, right.split(','))))
            # Append the plaintext and ciphertext to their respective lists
            plaintexts.append(u)
            ciphertexts.append(x)
        except Exception as e:
            # Print an error message if there is an issue parsing the line
            print(f"Error parsing the line: {line}")
            print(e)
    return plaintexts, ciphertexts

The following function is used to check if two array are equal. We will use it to check if plaintext (ciphertext) found with encryptor (decryptor) and the expected one are equal.

In [118]:
def test(result, target):
    print(result)
    if np.array_equal(result, target):
        print("Test passed")
    else:
        print("Test failed")

# Task 1 - Simplified AES-like cipher Encryption

This variables are used to test encryption and decryption of the first and second tasks

In [119]:
u = np.array([1, 0, 0, 0, 0, 0, 0, 0])
k = np.array([1, 0, 0, 0, 0, 0, 0, 0])

Then we define all functions that impement the linear AES system encryption

In [120]:
def subkey_generation(k):
    k1 = [k[0], k[2], k[4], k[6]]
    k2 = [k[0], k[1], k[2], k[3]]
    k3 = [k[0], k[3], k[4], k[7]]
    k4 = [k[0], k[3], k[5], k[6]]
    k5 = [k[0], k[2], k[5], k[7]]
    k6 = [k[2], k[3], k[4], k[5]]
    return np.array([
        k1, k2, k3, k4, k5, k6
    ])

In [121]:
def subkey_sum(w, ki):
    subkey = np.tile(ki, 2)  # k = ki + ki
    return (w + subkey[:len(w)]) % p

In [122]:
def substitution(v):
    # f = 2*v %p
    return (multiplicative_value_of_f * v) % p

In [123]:
def transposition(y):
    # flipped the second half of vector yi
    return np.array(y)[[0, 1, 2, 3, 7, 6, 5, 4]]

In [124]:
def linear(z):
    # linear transformation: write (by rows) vector z to 2 × 4 matrix Z
    Z = z.reshape(2, 4)
    w = (matrix_A @ Z) % p
    return w.flatten()

In [125]:
def encryption(u, k):
    subkey = subkey_generation(k)
    w = u.copy()
    for i in range(n):
        v = subkey_sum(w, subkey[i])
        y = substitution(v)
        z = transposition(y)
        if i != n-1:
            w = linear(z)
    x = subkey_sum(z, subkey[n])
    return x

Finally we can check if the encryption method defined above is correct.

In [126]:
x_test = [4, 0, 0, 9, 7, 0, 0, 3] # the expected result
x = encryption(u, k)
test(x, x_test)

[4 0 0 9 7 0 0 3]
Test passed


# Task 2 - Simplified AES-like cipher: Decryption

In [127]:
def inverse_subkey_sum(w, ki):
    k = np.tile(ki, 2)  # k = ki + ki
    return (w - k[:len(w)] + p) % p # Add p to avoid negative values

In [128]:
def inverse_substitution(v):
    inverse_of_multiplicative_value_of_f = pow(multiplicative_value_of_f, -1, p)
    return (v * inverse_of_multiplicative_value_of_f) % p

In [129]:
def inverse_transposition(y):
    return np.array(y)[[0, 1, 2, 3, 7, 6, 5, 4]]

In [130]:
def modular_inverse_matrix(A):
    # Compute the determinant of matrix A and round it to the nearest integer
    det = int(np.round(np.linalg.det(A)))
    # Compute the modular inverse of the determinant modulo p
    det_inv = pow(det, -1, p)
    # Compute the modular inverse of matrix A using the formula:
    # A^-1 % p = (det^-1 * adj(A)) mod p
    A_inv = np.round(det_inv * np.linalg.inv(A) * det)
    # Convert the result to integer type and apply modulo p
    return A_inv.astype(int) % p

In [131]:
def inverse_linear(z):
    # Compute the modular inverse of matrix_A
    A_inv = modular_inverse_matrix(matrix_A)
    # Reshape the input vector z into a 2x4 matrix
    w = z.reshape(2, 4)
    # Multiply A_inv with w and take the result modulo p
    w = (A_inv @ w) % p
    # Flatten the result back into a 1D array and return it
    return w.flatten()

In [132]:
def decryption(x, k):
    # Generate the list of round subkeys from the main key
    subkey = subkey_generation(k)
    # Undo the final key addition using the last subkey
    w = inverse_subkey_sum(x, subkey[n])
    # Iterate through the rounds in reverse order
    for i in range(n - 1, -1, -1):
        # Apply inverse linear transformation, except on the first iteration
        z = inverse_linear(w) if i != n - 1 else w
        # Apply inverse transposition step
        y = inverse_transposition(z)
        # Apply inverse substitution step
        v = inverse_substitution(y)
        # Subtract the round subkey (inverse of key addition)
        w = inverse_subkey_sum(v, subkey[i])
    # Return a copy of the decrypted output
    return w.copy()

Then, we are able to check the encryptor function. The value of `x` is one founded in previous task.

In [133]:
u = decryption(x, k)
u_test = [1, 0, 0, 0, 0, 0, 0, 0]
test(u, u_test)

[1 0 0 0 0 0 0 0]
Test passed


# Task 3 - Identify the cipher vulnerability

In [134]:
def generate_matrix_A_B():
    # Initialize matrices A and B with zeros
    A_matrix = np.zeros((dim, dim), dtype=int)
    B_matrix = np.zeros((dim, dim), dtype=int)

    # Compute matrix A: E(e_j, 0)
    for j in range(dim):
        # Create the unit vector e_j
        e_j = np.zeros(dim, dtype=int)
        e_j[j] = 1
        # Use e_j as the key and a zero vector as the input
        k = e_j
        u = np.zeros(dim, dtype=int)
        # Encrypt and store the result in the j-th column of A
        A_matrix[:, j] = encryption(u, k)

    # Compute matrix B: E(0, e_j)
    for j in range(dim):
        # Create the unit vector e_j
        e_j = np.zeros(dim, dtype=int)
        e_j[j] = 1
        # Use a zero vector as the key and e_j as the input
        k = np.zeros(dim, dtype=int)
        u = e_j
        # Encrypt and store the result in the j-th column of B
        B_matrix[:, j] = encryption(u, k)

    # Return the computed matrices A and B
    return A_matrix, B_matrix

In [135]:
def validate_matrixs(A, B, u, k):
    # x = E(k, u) = Ak + Bu mod p
    x_true = encryption(u, k)
    x_calculated = (A @ k + B @ u) % p

    # Check if the encryption property holds
    if np.array_equal(x_true, x_calculated):
        print("Test passed")
    else:
        print("Test failed")

In [136]:
A, B = generate_matrix_A_B()
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)
validate_matrixs(A, B, u, k)

Matrix A:
[[ 9  0  1  6  0  0  1 10]
 [ 0  8  6  2  2  9  0  0]
 [ 0  6  0  8  3 10  0  0]
 [ 6  0  0  8  0  1  6  6]
 [ 2  0  1 10  0  0  1  3]
 [ 0  1  8  4  9  6  0  0]
 [ 0 10  0  5  7  6  0  0]
 [ 3  0  0  1  0  1  4  8]]

Matrix B:
[[6 0 0 3 3 0 0 0]
 [0 6 3 0 0 3 0 0]
 [0 3 6 0 0 0 3 0]
 [3 0 0 6 0 0 0 3]
 [5 0 0 0 4 0 0 8]
 [0 5 0 0 0 4 8 0]
 [0 0 5 0 0 8 4 0]
 [0 0 0 5 8 0 0 4]]
Test passed


# Task 4 - Linear cryptanalysis KPA

In [137]:
def recover_key(u, x, A, B):
    # Compute the modular inverse of matrix A modulo p
    A_inv = modular_inverse_matrix(A)
    # Compute the product of matrix B and vector u modulo p
    Bu = (B @ u)
    # Compute the difference between the ciphertext x and Bu modulo p
    diff = (x - Bu)
    # Compute the key k using the formula: k = A_inv @ diff % p
    k = (A_inv @ diff) % p
    return k

In [138]:
def validateKey(plaintexts, ciphertexts, k):
    # Iterate over all plaintext-ciphertext pairs
    for i in range(len(plaintexts)):
        u = plaintexts[i]
        x = ciphertexts[i]
        # Compute the encryption of the plaintext u using the key k
        x_test = encryption(u, k)
        # Check if the computed ciphertext matches the given ciphertext
        if not np.array_equal(x, x_test):
            print("Test failed at index ", i)
            print("expected:" , x , "actual:" , x_test)
            return
    print("Test passed")

In [139]:
# Generate the matrices A and B using the function from task3
A, B = generate_matrix_A_B()

# Define the filepath to the file containing plaintext-ciphertext pairs
filepath = path + "KPApairsH_linear.txt"
# Read the plaintext-ciphertext pairs from the file
plaintexts, ciphertexts = read_pairs_from_file(filepath)

k = recover_key(plaintexts[0], ciphertexts[0], A, B)
print(k)

validateKey(plaintexts, ciphertexts, k)

[10  4 10  2  6  7  8  2]
Test passed


#Task 5 - “Nearly linear” simplified AES-like cipher

In [140]:
substitution_map = {0: 0, 1: 2, 2: 4, 3: 8, 4: 6, 5: 10, 6: 1, 7: 3, 8: 5, 9: 7, 10: 9}

In [141]:
def substitution_nearly_linear(v):
    # substitution: apply substitution map to vector v
    for i in range(len(v)):
        v[i] = substitution_map[v[i]]
    return v

In [142]:
def encryption_nearly_linear(u, k):
    subkey = subkey_generation(k)
    w = u.copy()
    for i in range(n):
        v = subkey_sum(w, subkey[i])
        y = substitution_nearly_linear(v)
        z = transposition(y)
        if i != n-1:
            w = linear(z)
    x = subkey_sum(z, subkey[n])
    return x

In [143]:
k = [1, 0, 0, 0, 0, 0, 0, 0]
u = [1, 0, 0, 0, 0, 0, 0, 0]
x_test = [9, 0, 0, 0, 5, 0, 0, 6] # the expected result
x = encryption_nearly_linear(u, k)
test(x, x_test)

[9 0 0 0 5 0 0 6]
Test passed


# Task 6 - Linear cryptanalysis of a “nearly linear” cipher

In [144]:
def near_keys(start_key, distance):
    # Dynamically determine the length of the key
    key_len = len(start_key)
    def backtrack(pos, current, remaining):
        # If we've filled all positions
        if pos == key_len:
            # Only yield if the total distance is exactly matched
            if remaining == 0:
                yield (start_key + np.array(current)) % p
            return

        # Explore all possible deltas within the remaining distance
        for delta in range(-remaining, remaining + 1):
            current[pos] = delta
            # Recurse with updated position and reduced remaining distance
            yield from backtrack(pos + 1, current, remaining - abs(delta))

    # Initialize a zero-delta key to start from
    current = [0] * key_len
    yield from backtrack(0, current, distance)

In [145]:
def key_test(k, plaintexts, ciphertexts):
    if not np.array_equal(encryption_nearly_linear(plaintexts[0], k), ciphertexts[0]):
        return False
    for u, x in zip(plaintexts, ciphertexts):
        if not np.array_equal(encryption_nearly_linear(u, k), x):
            return False
    return True

In [146]:
def brute_force(start_key, plaintexts, ciphertexts, max_distance):
    # Iterate through increasing distances from the start_key
    for distance in range(1, max_distance + 1):
        count = 0
        print(f"Testing keys at distance {distance}")
        # Generate all keys within the current distance
        for key in near_keys(start_key, distance):
            count += 1
            # Check if the current key correctly encrypts all plaintexts
            if key_test(key, plaintexts, ciphertexts):
                return key  # Found a matching key
        # Print how many keys were tested at this distance
        print(count, "tested keys at distance", distance)
    # No matching key found within the given max distance
    return None

In [147]:
def recover_one_key(u, x, A, B, C):
    # Compute the modular inverse of matrix A
    A_inv = modular_inverse_matrix(A)
    # Compute the product of matrix B and vector u
    Bu = (B @ u)
    # Compute the product of matrix C and vector x
    Cx = (C @ x)
    # Compute the difference between the Cx and Bu
    diff = (-Cx - Bu)
    # Compute the key k using the formula: k = A_inv @ diff % p
    k = (A_inv @ diff) % p
    return k

In [148]:
def key_approximation(A, B, C, plaintexts, ciphertexts):
    # Recover one possible key for each plaintext-ciphertext pair
    k = []
    for i in range(len(plaintexts)):
        u = plaintexts[i]
        x = ciphertexts[i]
        k.append(recover_one_key(u, x, A, B, C))
    num_keys = len(k)         # Total number of recovered key candidates
    key_length = len(k[0])    # Length of each key vector

    # Initialize list to store the average of each key position
    averages = []
    # Compute the average of each key component across all recovered keys
    for j in range(key_length):
        avg = sum(k[i][j] for i in range(num_keys)) / num_keys
        averages.append(round(avg))  # Round to nearest integer

    # Return the approximated key
    return averages

In [149]:
def zero_probability(k, A, B, C):
    count_zeros = 0
    count_all = 0
    #generate a vector of 100k random vectors of 8 bit with values between 0 and 10
    for i in range(100000):
        u = np.random.randint(0, p, size=dim)
        count_all += 1
        if is_prob_zero(u, k, A, B, C):
            count_zeros += 1
    return count_zeros / count_all
    

def is_prob_zero(u, k, A, B, C):
    x = encryption_nearly_linear(u, k)
    result = (A @ k + B @ u - C @ x)%p
    return np.all(result == 0)

In [150]:
filepath = path + "KPApairsH_nearly_linear.txt"
plaintexts, ciphertexts = read_pairs_from_file(filepath)
A, B = generate_matrix_A_B()
C = np.identity(dim)
key = key_approximation(A, B, C, plaintexts, ciphertexts)
print("Key approximation: ", key)
print("P[Ak+Bu+Cx %p = 0]:", zero_probability(key, A, B, C))
print("1/(p^lx):", 1/(p**dim))

Key approximation:  [5, 4, 4, 8, 2, 6, 3, 4]
P[Ak+Bu+Cx %p = 0]: 0.00034
1/(p^lx): 4.665073802097334e-09


In [151]:
final_key = brute_force(key, plaintexts, ciphertexts, max_distance)
if final_key is not None:
    print("Found key", final_key)
else:
    print("No valid key found")

Testing keys at distance 1
16 tested keys at distance 1
Testing keys at distance 2
128 tested keys at distance 2
Testing keys at distance 3
688 tested keys at distance 3
Testing keys at distance 4
2816 tested keys at distance 4
Testing keys at distance 5
9424 tested keys at distance 5
Testing keys at distance 6
27008 tested keys at distance 6
Testing keys at distance 7
68464 tested keys at distance 7
Testing keys at distance 8
157184 tested keys at distance 8
Testing keys at distance 9
332688 tested keys at distance 9
Testing keys at distance 10
658048 tested keys at distance 10
Testing keys at distance 11
1229360 tested keys at distance 11
Testing keys at distance 12
2187520 tested keys at distance 12
Testing keys at distance 13
Found key [ 5 10  4  9  2  3  3  8]


# Task 7 - Non linear simplified AES-like cipher

In [152]:
u = np.array([1, 0, 0, 0, 0, 0, 0, 0])
k = np.array([1, 0, 0, 0])

In [153]:
def subkey_generation_non_linear(k):
    k1 = [k[0], k[1], k[2], k[3]]
    k2 = [k[0], k[1], k[3], k[2]]
    k3 = [k[1], k[2], k[3], k[0]]
    k4 = [k[0], k[3], k[1], k[2]]
    k5 = [k[2], k[3], k[0], k[1]]
    k6 = [k[1], k[3], k[0], k[2]]
    return np.array([
        k1, k2, k3, k4, k5, k6
    ])

In [154]:
def modular_inverse(v):
    # Compute the modular multiplicative inverse modulo p for each element in v
    inverses = []
    for x in v:
        # In our case, p = 11; the only non-invertible value is 0
        if np.gcd(x, p) == 1:
            inverses.append(pow(int(x), -1, p))  # Compute inverse modulo p
        else:
            inverses.append(0)  # Assign 0 if not invertible (e.g., x = 0)
    return np.array(inverses)

In [155]:
def substitution_non_linear(v):
    v_inv = modular_inverse(v)
    return (multiplicative_value_of_f * v_inv) % p  # Apply the substitution

In [156]:
def encryption_non_linear(u, k):
    subkey = subkey_generation_non_linear(k)
    w = u.copy()
    for i in range(n):
        v = subkey_sum(w, subkey[i])
        y = substitution_non_linear(v)
        z = transposition(y)
        if i != n-1:
            w = linear(z)
    x = subkey_sum(z, subkey[n])
    return x

We also define the functions to decrypt (not required but useful for next task).

In [157]:
def inverse_substitution_non_linear(y):
    y_inv = (y * pow(multiplicative_value_of_f, -1, p)) % p  # Reverse substitution step
    return modular_inverse(y_inv)  # Apply modular inverse

In [158]:
def decryption_non_linear(x, k):
    subkey = subkey_generation_non_linear(k)
    w = inverse_subkey_sum(x, subkey[n])  # Undo the last subkey sum
    for i in range(n - 1, -1, -1):  # Reverse the encryption rounds
        z = inverse_linear(w) if i != n - 1 else w
        y = inverse_transposition(z)
        v = inverse_substitution_non_linear(y)
        w = inverse_subkey_sum(v, subkey[i])
    return w.copy()

In [159]:
x_test = [5, 0, 3, 2, 5, 2, 1, 1]
x = encryption_non_linear(u, k)
test(x, x_test)
u = decryption_non_linear(x, k)
test(u, u_test)

[5 0 3 2 5 2 1 1]
Test passed
[1 0 0 0 0 0 0 0]
Test passed


# Task 8 - Meet in the middle attack

In [160]:
dim = 4
n_guesses = 50000

In [161]:
def generate_random_key(dim, p):
    return np.array([random.randint(0, p-1) for _ in range(dim)])

In [162]:
def generate_key_guesses(n_guesses):
    key_prime = [generate_random_key(dim, p) for _ in range(n_guesses)]
    key_second = [generate_random_key(dim, p) for _ in range(n_guesses)]
    return key_prime, key_second

In [163]:
def generate_plaintext_ciphertext_from_keys(u, x, keys_prime, keys_second):
    ciphertexts = {}
    plaintexts = {}
    for kp in keys_prime:
        ciphertexts[tuple(kp)] = encryption_non_linear(u, kp)
    for ks in keys_second:
        plaintexts[tuple(ks)] = decryption_non_linear(x, ks)
    return ciphertexts, plaintexts

In [164]:
def look_for_matches(generated_ciphertexts, generated_plaintexts):
    matching_pairs = []
    for ks in generated_ciphertexts.keys():
        for kp in generated_plaintexts.keys():
            if np.array_equal(generated_ciphertexts[ks], generated_plaintexts[kp]):
                matching_pairs.append([{ks: generated_ciphertexts[ks]}, {kp: generated_plaintexts[kp]}])
    return matching_pairs

In [165]:
def key_works_for_all_pair(plaintexts, ciphertexts, key):
    k1 = list(key[0].keys())[0]  # Get the key from the first dictionary in the list
    k2 = list(key[1].keys())[0]  # Get the key from the second dictionary in the list

    for i in range(len(plaintexts)):
        u = plaintexts[i]
        x = ciphertexts[i]

        encrypted_with_k1 = encryption_non_linear(u, k1)
        encrypted_with_k2 = encryption_non_linear(encrypted_with_k1, k2)
        if np.not_equal(encrypted_with_k2, x).all():
            return None
        decrypted_with_k2 = decryption_non_linear(x, k2)
        decrypted_with_k1 = decryption_non_linear(decrypted_with_k2, k1)
        if np.not_equal(decrypted_with_k1, u).all():
            return None
    return k1, k2

In [166]:
# Define the filepath to the file containing plaintext-ciphertext pairs
filepath = path + "KPApairsH_non_linear.txt"

# Read the plaintext-ciphertext pairs from the file
original_plaintexts, original_ciphertexts = read_pairs_from_file(filepath)

# Generate random key guesses
keys_prime, keys_second = generate_key_guesses(n_guesses)

# Generate encryption & decryption mappings
generated_ciphertexts, generated_plaintexts = generate_plaintext_ciphertext_from_keys(
    original_plaintexts[0],
    original_ciphertexts[0],
    keys_prime,
    keys_second
)

# Look for key matches
results = look_for_matches(generated_ciphertexts, generated_plaintexts)

print(f"Found {len(results)} matching key pairs:")
for match in results:
    result = key_works_for_all_pair(original_plaintexts, original_ciphertexts, match)
    if result is not None:
        k1, k2 = result
        k1_str = "[" + ",".join(str(x) for x in k1) + "]"
        k2_str = "[" + ",".join(str(x) for x in k2) + "]"
        print(f"Found the following key that works with all pairs of given plaintext-ciphertext:\nk1={k1_str}\nk2={k2_str}")

Found 2 matching key pairs:
Found the following key that works with all pairs of given plaintext-ciphertext:
k1=[7,7,3,8]
k2=[0,0,10,1]
